In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import json
import os
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from src.func import tweet_utils
from src.func import regex
from src.func import labmtgen
from src.sentiment.senti_utils import *
from labMTsimple.storyLab import *
import random
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt

In [5]:
tweet_dir= Path("../data/processed/tweets/")
city = Path("../data/processed/tweets/CA_San_Diego_0666000.json")

In [25]:
lang = 'english'
labMT,labMTvector,labMTwordList = emotionFileReader(stopval=1.0,lang=lang,returnVector=True)
# Find Most influential words using labmt shift method

for city in tweet_dir.glob("*.json"):
    city_name = city.stem
    if city_name not in old_city:
        print("{}".format(city_name))
        PVec, CVec,pStops = getVecs(city)
        PVec, Pval = labmtSimpleSentiment(PVec,pStops)
        CVec, Cval = labmtSimpleSentiment(CVec,pStops)
        sortedMag,sortedWords,sortedType,sumTypes = shift(CVec, PVec,labMTvector, labMTwordList,sort=True)
        city_words = []
        for i in range(50):
            word = sortedWords[i]
            entry = {"word":word, "valence":labMTvector[labMTwordList.index(word)], "contribution":sortedMag[i]}
            city_words.append(entry)
        city_df = pd.DataFrame.from_records(city_words, index='word')
        with pd.ExcelWriter('./results/city_words.xlsx', mode='a') as writer:
            city_df.to_excel(writer, sheet_name=city.stem)

CO_Denver_0820000
TX_Fort_Worth_4827000
TX_El_Paso_4824000
TN_Memphis_4748000
MI_Detroit_2622000
NC_Charlotte_3712000
IN_Indianapolis_1836003
OH_Columbus_3918000
MD_Baltimore_2404000


In [40]:
xls = pd.ExcelFile('./results/city_words.xlsx')
sheets =  pd.read_excel(xls, None)

In [45]:
words_investigate = {}
for city, df in sheets.items():
    words_investigate[city] = list(df[df.remove ==1.0].word.values)

In [47]:
words_investigate

{'CA_San_Francisco_0667000': ['festival', 'music', 'young', 'flowers'],
 'AZ_Phoenix_0455000': ['garden', 'hospital', 'zoo'],
 'FL_Jacksonville_1235000': ['science', 'museum'],
 'TX_Austin_4805000': ['music', 'limits', 'accident'],
 'CA_San_Diego_0666000': ['zoo', 'sea'],
 'DC_Washington_1150000': ['war', 'bill', 'united', 'health'],
 'WA_Seattle_5363000': ['health', 'surgery', 'emergency'],
 'IL_Chicago_1714000': ['museum', 'music', 'art', 'festival', 'riot'],
 'TX_Houston_4835000': ['hospital',
  'traffic',
  'delay',
  'stop',
  'accident',
  'science',
  'museum'],
 'OH_Cleveland_3916000': ['garden', 'beach', 'island'],
 'MA_Boston_2507000': ['closed', 'ma', 'partners', 'festival'],
 'NY_New_York_3651000': ['festival', 'natural'],
 'TX_San_Antonio_4865000': ['cafe', 'zoo'],
 'TX_Dallas_4819000': ['festival', 'music', 'health'],
 'PA_Philadelphia_4260000': ['independence', 'art'],
 'CA_Los_Angeles_0644000': ['zoo', 'science', 'festival'],
 'CA_San_Jose_0668000': ['christmas', 'ragin